# Analiza sportowa - Czyszczenie danych oraz ich przygotowanie do analizy
## Projekt realizowany w ramach przedmiotu: Metody Przetwarzania Dużych Ilości Danych
### Grupa projektowa:
+  Filip Kozak
+  Maciej Grzela
+  Paweł Owczarek
+  Szymon Tusznio

## W ramach niniejszego etapu przeprowadzone zostaną następujące operacje:
+ ### Wczytanie danych z pliku CSV
+ ### Usunięcie wpisów niepoprawnych z punktu widzenia typu danych
+ ### Usunięcie wpisów niepoprawnych z punktu widzenia wiedzy dziedzinowej
+ ### Modyfikacja formatu przechowywania wartości liczbowych
+ ### Wyliczenie wartości dla danych pustych

### Zaimportowanie wykorzystywanych bibliotek oraz konfiguracja wyświetlania zbiorów danych dla biblioteki Pandas

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 320)
pd.reset_option("display.max_rows")

# pd.set_option("display.max_rows", None)           # Wyświetlenie wszystkich wierszy dla tabel / spowalnia działanie
css = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(css))

### Wczytanie danych z pliku csv oraz zapisanie ich do struktury DataFrame dostępnej w ramach biblioteki Pandas
### Sprawdzenie dla jakich kolumn występują wartości nieokreślone

In [2]:
transfer_df = pd.read_csv(
    'csv/transfers-data-frame.csv',
    sep=',',
    header=0,
    index_col=0,
    parse_dates=['TransferDate']
)

display(transfer_df)
display(pd.isnull(transfer_df).sum())

,Club,ClubLeagueLevel,ClubLeagueNationality,Player,PlayerAge,PlayerHeight,TransferDate,PlayerMatchesPlayed,PlayerGoals,PlayerAssists,PlayerOwnGoals,PlayerSubstitutionIn,PlayerSubstitutionOut,PlayerYellowCards,PlayerSecondYellowCards,PlayerRedCards,PlayerMinutesPlayed,PlayerPosition,PlayerPositionScut,PlayerNationality,Operation,Bought/Sold,PlayerMarketValue,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue
0,Legia Warszawa,1.liga,Polska,Ernest Muci,19,1.80,23 lut 2021,94,24,6,0,41,26,6,0,0,5.004',Środkowy napastnik,ŚNP,Albania,500 tys. €,Nabytki,"1,00 mln €",KF Tirana,1.liga,Albania,500 tys. €
1,Legia Warszawa,1.liga,Polska,Josip Juranovic,24,1.73,21 sie 2021,224,8,23,3,23,12,18,0,1,18.428',Prawy obrońca,POB,Chorwacja,400 tys. €,Nabytki,"2,50 mln €",HNK Hajduk Split,1.liga,Chorwacja,400 tys. €
2,Legia Warszawa,1.liga,Polska,Jasur Yakhshiboev,23,1.80,30 sie 2021,103,23,13,0,50,31,14,0,0,5.086',Prawy napastnik,PNP,Uzbekistan,300 tys. €,Nabytki,700 tys. €,Pakhtakor Tashkent,1.liga,Uzbekistan,300 tys. €
3,Legia Warszawa,1.liga,Polska,Rafa Lopes,29,1.85,1 sie 2020,375,75,19,1,123,120,41,0,1,23.321',Ofensywny pomocnik,OPM,Portugalia,150 tys. €,Nabytki,500 tys. €,Cracovia,1.liga,Polska,150 tys. €
4,Legia Warszawa,1.liga,Polska,Bartosz Kapustka,23,1.79,13 sie 2020,151,20,19,0,36,54,25,0,1,10.003',Ofensywny pomocnik,OPM,Polska,bez odstępnego,Nabytki,"1,50 mln €",Leicester City,1.liga,Anglia,bez odstępnego
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27105,Bank El Ahly,1.liga,Egipt,Shady Radwan,19,1.73,6 wrz 2021,7,0,0,0,7,1,1,0,0,71',Prawy napastnik,PNP,Egipt,koniec wypżyczenia31 sie 2021,Odejścia,25 tys. €,El Ahly Kairo U21,NaN,NaN,koniec wypżyczenia31 sie 2021
27106,Bank El Ahly,1.liga,Egipt,Ismail El Leithy,21,NaN,28 sty 2019,0,0,0,0,0,0,0,0,0,0,Środkowy pomocnik,ŚPM,Egipt,koniec wypżyczenia31 sie 2021,Odejścia,25 tys. €,El Ahly Kairo U21,NaN,NaN,koniec wypżyczenia31 sie 2021
27107,Bank El Ahly,1.liga,Egipt,Karim Bambo,28,1.75,2 wrz 2021,154,38,11,0,92,50,6,0,0,6.527',Lewy napastnik,LNP,Egipt,koniec wypżyczenia31 sie 2021,Odejścia,500 tys. €,Zamalek SC,1.liga,Egipt,koniec wypżyczenia31 sie 2021
27108,Bank El Ahly,1.liga,Egipt,Mohamed Fathi,27,1.78,12 wrz 2021,171,5,2,0,28,40,29,4,2,12.329',Defensywny pomocnik,DPM,Egipt,koniec wypżyczenia31 sie 2021,Odejścia,350 tys. €,Pyramids FC,1.liga,Egipt,koniec wypżyczenia31 sie 2021


Club                                0
ClubLeagueLevel                     0
ClubLeagueNationality               0
Player                              0
PlayerAge                           0
PlayerHeight                     1930
TransferDate                     1857
PlayerMatchesPlayed                 0
PlayerGoals                         0
PlayerAssists                       0
PlayerOwnGoals                      0
PlayerSubstitutionIn                0
PlayerSubstitutionOut               0
PlayerYellowCards                   0
PlayerSecondYellowCards             0
PlayerRedCards                      0
PlayerMinutesPlayed                 0
PlayerPosition                      0
PlayerPositionScut                  0
PlayerNationality                   0
Operation                           0
Bought/Sold                         0
PlayerMarketValue                   0
ClubAssigned                      233
ClubAssignedLeagueLevel          5772
ClubAssignedLeagueNationality    5772
TransferValu

### Wyświetlenie wierszy dla których występują wartości nieokreślone

In [3]:
is_nan = transfer_df.isna()
rows_has_nan = is_nan.any(axis=1)
transfer_df[rows_has_nan][["Club", "Player", "PlayerHeight", "TransferDate", "ClubAssigned", "ClubAssignedLeagueLevel", "ClubAssignedLeagueNationality", "TransferValue"]]

,Club,Player,PlayerHeight,TransferDate,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue
35,Legia Warszawa,Dominik Nagy,1.74,20 sty 2021,Bez klubuBez klubu,NaN,NaN,-
36,Legia Warszawa,William Rémy,1.86,NaN,Bez klubuBez klubu,NaN,NaN,-
43,Lech Poznań,Aron Jóhannsson,1.84,NaN,Hammarby IF,1.liga,Szwecja,bez odstępnego
59,Lech Poznań,Bogdan Butko,1.82,NaN,Szachtar Donieck,1.liga,Ukraina,wypożyczenie
85,Lech Poznań,Bogdan Butko,1.82,NaN,Szachtar Donieck,1.liga,Ukraina,koniec wypżyczenia4 sty 2021
...,...,...,...,...,...,...,...,...
27102,Bank El Ahly,Mohamed Ateya,1.83,NaN,Bez klubuBez klubu,NaN,NaN,-
27103,Bank El Ahly,Hamad El Hanbaly,1.86,NaN,Bez klubuBez klubu,NaN,NaN,-
27104,Bank El Ahly,Mohamed Fakhri,1.85,16 sty 2018,El Ahly Kairo U21,NaN,NaN,koniec wypżyczenia31 sie 2021
27105,Bank El Ahly,Shady Radwan,1.73,6 wrz 2021,El Ahly Kairo U21,NaN,NaN,koniec wypżyczenia31 sie 2021


### Wnioskując na podstawie wierszy zawierających wartości nieokreślone możemy zauważyć, że przeprowadzane transfery dotyczą:
+ ### Przenosin zawodnika do drużyny rezerw/drużyny młodzieżowej/szkółki piłkarskiej
+ ### Przenosin zawodnika do drużyny na poziomie rozgrywkowym nieuwzględnianym w ramach grupy poziomów rozgrywkowych futbolu profesjonalnego
+ ### Dyskwalifikacji/pauzy zawodnika
+ ### Przenosin zawodnika do klubu piłkarskiego, którego dane nie zostały wprowadzone
+ ### Zakończenia współpracy pomiędzy zawodnikiem a klubem

### Obserwacje spełniające jedno z czterech pierwszych kryteriów powyższej listy możemy uznać jako obserwacje odstające. Wiersze te, zostaną usunięte ze zbioru danych.
### Obserwacje spełniające piąte kryterium należy uznać jako poprawne. Aby uwzględnić przypadek zakończenia współpracy pomiędzy zawodnikiem a klubem należy stworzyć nową kolumnę wartości True/False

### Zdefiniowanie kolumny 'PlayerWithoutClub' przyjmującej wartość true dla zawodników bez klubu po transferze
### Wyczyszczenie wartości dla kolumny 'ClubAssigned'

In [4]:
without_club_label = "Bez Klubu"

transfer_df["ClubAssigned"].replace({"Bez klubuBez klubu": without_club_label}, inplace=True)
transfer_df['PlayerWithoutClub'] = transfer_df['ClubAssigned'].map(lambda x: x == True if without_club_label else False)

transfer_df[rows_has_nan][["Club", "Player", "PlayerHeight", "TransferDate", "ClubAssigned", "ClubAssignedLeagueLevel", "ClubAssignedLeagueNationality", "TransferValue"]]

,Club,Player,PlayerHeight,TransferDate,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue
35,Legia Warszawa,Dominik Nagy,1.74,20 sty 2021,Bez Klubu,NaN,NaN,-
36,Legia Warszawa,William Rémy,1.86,NaN,Bez Klubu,NaN,NaN,-
43,Lech Poznań,Aron Jóhannsson,1.84,NaN,Hammarby IF,1.liga,Szwecja,bez odstępnego
59,Lech Poznań,Bogdan Butko,1.82,NaN,Szachtar Donieck,1.liga,Ukraina,wypożyczenie
85,Lech Poznań,Bogdan Butko,1.82,NaN,Szachtar Donieck,1.liga,Ukraina,koniec wypżyczenia4 sty 2021
...,...,...,...,...,...,...,...,...
27102,Bank El Ahly,Mohamed Ateya,1.83,NaN,Bez Klubu,NaN,NaN,-
27103,Bank El Ahly,Hamad El Hanbaly,1.86,NaN,Bez Klubu,NaN,NaN,-
27104,Bank El Ahly,Mohamed Fakhri,1.85,16 sty 2018,El Ahly Kairo U21,NaN,NaN,koniec wypżyczenia31 sie 2021
27105,Bank El Ahly,Shady Radwan,1.73,6 wrz 2021,El Ahly Kairo U21,NaN,NaN,koniec wypżyczenia31 sie 2021


In [5]:
nans_with_club = transfer_df[rows_has_nan].loc[transfer_df['PlayerWithoutClub'] == False]
nans_with_height = transfer_df[rows_has_nan].loc[transfer_df['PlayerHeight'] == np.nan]
nans_with_date = transfer_df[rows_has_nan].loc[transfer_df['TransferDate'] == np.nan]

nans_for_height_or_date = pd.merge(nans_with_height, nans_with_date, how='inner')

transfer_df.drop(nans_with_club.index.tolist(), inplace=True)
transfer_df.drop(nans_for_height_or_date.index.tolist(), inplace=True)

display(transfer_df)

,Club,ClubLeagueLevel,ClubLeagueNationality,Player,PlayerAge,PlayerHeight,TransferDate,PlayerMatchesPlayed,PlayerGoals,PlayerAssists,PlayerOwnGoals,PlayerSubstitutionIn,PlayerSubstitutionOut,PlayerYellowCards,PlayerSecondYellowCards,PlayerRedCards,PlayerMinutesPlayed,PlayerPosition,PlayerPositionScut,PlayerNationality,Operation,Bought/Sold,PlayerMarketValue,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue,PlayerWithoutClub
0,Legia Warszawa,1.liga,Polska,Ernest Muci,19,1.80,23 lut 2021,94,24,6,0,41,26,6,0,0,5.004',Środkowy napastnik,ŚNP,Albania,500 tys. €,Nabytki,"1,00 mln €",KF Tirana,1.liga,Albania,500 tys. €,False
1,Legia Warszawa,1.liga,Polska,Josip Juranovic,24,1.73,21 sie 2021,224,8,23,3,23,12,18,0,1,18.428',Prawy obrońca,POB,Chorwacja,400 tys. €,Nabytki,"2,50 mln €",HNK Hajduk Split,1.liga,Chorwacja,400 tys. €,False
2,Legia Warszawa,1.liga,Polska,Jasur Yakhshiboev,23,1.80,30 sie 2021,103,23,13,0,50,31,14,0,0,5.086',Prawy napastnik,PNP,Uzbekistan,300 tys. €,Nabytki,700 tys. €,Pakhtakor Tashkent,1.liga,Uzbekistan,300 tys. €,False
3,Legia Warszawa,1.liga,Polska,Rafa Lopes,29,1.85,1 sie 2020,375,75,19,1,123,120,41,0,1,23.321',Ofensywny pomocnik,OPM,Portugalia,150 tys. €,Nabytki,500 tys. €,Cracovia,1.liga,Polska,150 tys. €,False
4,Legia Warszawa,1.liga,Polska,Bartosz Kapustka,23,1.79,13 sie 2020,151,20,19,0,36,54,25,0,1,10.003',Ofensywny pomocnik,OPM,Polska,bez odstępnego,Nabytki,"1,50 mln €",Leicester City,1.liga,Anglia,bez odstępnego,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27084,Bank El Ahly,1.liga,Egipt,Mohamed Fathi,26,1.78,12 wrz 2021,171,5,2,0,28,40,29,4,2,12.329',Defensywny pomocnik,DPM,Egipt,wypożyczenie,Nabytki,350 tys. €,Pyramids FC,1.liga,Egipt,wypożyczenie,False
27085,Bank El Ahly,1.liga,Egipt,Karim Bambo,27,1.75,2 wrz 2021,154,38,11,0,92,50,6,0,0,6.527',Lewy napastnik,LNP,Egipt,wypożyczenie,Nabytki,500 tys. €,Zamalek SC,1.liga,Egipt,wypożyczenie,False
27107,Bank El Ahly,1.liga,Egipt,Karim Bambo,28,1.75,2 wrz 2021,154,38,11,0,92,50,6,0,0,6.527',Lewy napastnik,LNP,Egipt,koniec wypżyczenia31 sie 2021,Odejścia,500 tys. €,Zamalek SC,1.liga,Egipt,koniec wypżyczenia31 sie 2021,False
27108,Bank El Ahly,1.liga,Egipt,Mohamed Fathi,27,1.78,12 wrz 2021,171,5,2,0,28,40,29,4,2,12.329',Defensywny pomocnik,DPM,Egipt,koniec wypżyczenia31 sie 2021,Odejścia,350 tys. €,Pyramids FC,1.liga,Egipt,koniec wypżyczenia31 sie 2021,False


### Spośród pozostałych wierszy zawierających wartości nieokreślone wszystkie posiadają dla kolumny 'PlayerWithoutClub' wartość True
### Wiersze te nie zostaną usunięte, dlatego też należy usunąć z nich wartości NaN i zastąpić je pustym ciągiem znaków

In [6]:
transfer_df.fillna('', inplace=True)

### Jak możemy zauważyć, po wykonaniu powyższej czynności w zbiorze danych nie występują już żadne wartości nieokreślone

In [7]:
transfer_df[transfer_df.isna().any(axis=1)]

,Club,ClubLeagueLevel,ClubLeagueNationality,Player,PlayerAge,PlayerHeight,TransferDate,PlayerMatchesPlayed,PlayerGoals,PlayerAssists,PlayerOwnGoals,PlayerSubstitutionIn,PlayerSubstitutionOut,PlayerYellowCards,PlayerSecondYellowCards,PlayerRedCards,PlayerMinutesPlayed,PlayerPosition,PlayerPositionScut,PlayerNationality,Operation,Bought/Sold,PlayerMarketValue,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue,PlayerWithoutClub


### Wyczyszczenie danych z kolumn 'ClubLeagueLevel' oraz 'ClubAssignedLeagueLevel'
### Dla ligi młodzieżowej oraz rezerw klubowych założono najniższy (8smy) poziom rozgrywkowy, natomiast dla zawodników bez klubu przyjęto poziom ligi jako 0

In [8]:
transfer_df["ClubLeagueLevel"] = transfer_df['ClubLeagueLevel'].map(lambda x: x[0])
display(transfer_df['ClubLeagueLevel'].value_counts())

transfer_df["ClubAssignedLeagueLevel"].replace({"ieżowa": "8.liga", "onship": "2.liga", "rezerw": "8.liga", "": "0.liga"}, inplace=True)
transfer_df["ClubAssignedLeagueLevel"] = transfer_df['ClubAssignedLeagueLevel'].map(lambda x: x[0])
display(transfer_df['ClubAssignedLeagueLevel'].value_counts())

transfer_df['ClubLeagueLevel'] = transfer_df['ClubLeagueLevel'].astype("int64")
transfer_df['ClubAssignedLeagueLevel'] = transfer_df['ClubAssignedLeagueLevel'].astype("int64")

transfer_df.info()

1    17750
2      867
3      866
Name: ClubLeagueLevel, dtype: int64

1    11157
2     5376
3     1478
8      662
4      648
5      107
6       55
Name: ClubAssignedLeagueLevel, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19483 entries, 0 to 27109
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Club                           19483 non-null  object 
 1   ClubLeagueLevel                19483 non-null  int64  
 2   ClubLeagueNationality          19483 non-null  object 
 3   Player                         19483 non-null  object 
 4   PlayerAge                      19483 non-null  object 
 5   PlayerHeight                   19483 non-null  float64
 6   TransferDate                   19483 non-null  object 
 7   PlayerMatchesPlayed            19483 non-null  int64  
 8   PlayerGoals                    19483 non-null  int64  
 9   PlayerAssists                  19483 non-null  int64  
 10  PlayerOwnGoals                 19483 non-null  int64  
 11  PlayerSubstitutionIn           19483 non-null  int64  
 12  PlayerSubstitutionOut          19483 non-null 

### Wyświetlenie zgrupowanych wartości dla kolumn w celu określenia, w jaki sposób powinny być czyszczone

In [9]:
display(transfer_df['ClubLeagueNationality'].value_counts())         # nie wymaga czyszczenia
display(transfer_df['PlayerAge'].value_counts())                     # nie wymaga czyszczenia
display(transfer_df['PlayerHeight'].value_counts())                  # nie wymaga czyszczenia
display(transfer_df['TransferDate'].value_counts())                  # usunięcie wierszy dla wartości zero / konwersja daty
display(transfer_df['PlayerMatchesPlayed'].value_counts())           # nie wymaga czyszczenia
display(transfer_df['PlayerGoals'].value_counts())                   # nie wymaga czyszczenia
display(transfer_df['PlayerAssists'].value_counts())                 # nie wymaga czyszczenia
display(transfer_df['PlayerOwnGoals'].value_counts())                # nie wymaga czyszczenia
display(transfer_df['PlayerSubstitutionIn'].value_counts())          # nie wymaga czyszczenia
display(transfer_df['PlayerSubstitutionOut'].value_counts())         # nie wymaga czyszczenia
display(transfer_df['PlayerYellowCards'].value_counts())             # nie wymaga czyszczenia
display(transfer_df['PlayerSecondYellowCards'].value_counts())       # nie wymaga czyszczenia
display(transfer_df['PlayerRedCards'].value_counts())                # nie wymaga czyszczenia
display(transfer_df['PlayerMinutesPlayed'].value_counts())           # usunięcie apostrofu oraz znaku kropki, a także konwersja na typ liczbowy
display(transfer_df['PlayerPosition'].value_counts())                # nie wymaga czyszczenia (stworzenie dodatkowej kolumny danych określających pozycję ogólną - Pomocnik, Napastnik, Obrońca, Bramkarz
display(transfer_df['PlayerNationality'].value_counts())             # nie wymaga czyszczenia
display(transfer_df['Operation'].value_counts())                     # wymaga czyszczenia (usunięcie danych niepoprawnych oraz modyfikacja formatu numerycznego, usunięcie wierszy dla wartości '?'
display(transfer_df['Bought/Sold'].value_counts())                   # zmiana wartości 'nabytki' na 'kupiony' oraz 'odejścia' na 'sprzedany'
display(transfer_df['PlayerMarketValue'].value_counts())             # modyfikacja danych do wartości liczbowych
display(transfer_df['ClubAssignedLeagueNationality'].value_counts()) # nie wymaga czyszczenia
display(transfer_df['TransferValue'].value_counts())                 # analogicznie do kolumny 'Operation'

Anglia                          2379
Turcja                           968
Włochy                           886
Brazylia                         801
Argentyna                        799
Portugalia                       656
Stany Zjednaczone                639
Serbia                           638
Hiszpania                        623
Meksyk                           612
Japonia                          609
Francja                          561
Czechy                           559
Holandia                         553
Belgia                           552
Rosja                            544
Polska                           526
Niemcy                           510
Kolumbia                         489
Grecja                           473
Chorwacja                        403
Bułgaria                         362
Chiny                            362
Norwegia                         348
Węgry                            335
Szwecja                          331
Cypr                             326
A

23    1902
21    1780
22    1733
24    1642
20    1539
25    1522
26    1482
27    1309
28    1125
29    1081
19     956
30     766
31     687
32     511
18     425
33     352
34     199
17     155
35     139
36      77
16      36
37      30
38      16
39       8
40       6
15       3
-        1
41       1
Name: PlayerAge, dtype: int64

1.80    1399
1.85    1222
1.83    1144
1.78    1134
1.75    1027
1.88     955
1.86     929
1.84     897
1.82     879
1.87     862
1.90     790
1.81     789
1.77     744
1.76     682
1.79     633
1.73     590
1.74     570
1.91     485
1.89     473
1.72     431
1.70     426
1.93     356
1.92     308
1.71     282
1.68     210
1.95     196
1.94     174
1.69     158
1.96     158
1.67     137
1.65      79
1.97      62
1.66      61
1.98      53
1.64      46
1.63      29
1.99      20
2.01      19
1.60      14
2.02      14
1.62      13
2.00       8
2.03       8
1.55       6
1.61       3
1.56       2
2.04       2
1.57       2
1.58       1
1.54       1
Name: PlayerHeight, dtype: int64

1 lip 2021     2038
1 lip 2020      702
1 sty 2021      605
31 sie 2021     550
1 lut 2021      460
               ... 
5 lut 2020        1
11 lip 2017       1
16 lut 2017       1
16 lis 2017       1
6 gru 2021        1
Name: TransferDate, Length: 810, dtype: int64

74     117
119    113
126    109
103    107
94     105
      ... 
446      1
723      1
526      1
464      1
515      1
Name: PlayerMatchesPlayed, Length: 584, dtype: int64

...

bez odstępnego                       5368
wypożyczenie                         4638
?                                    1758
koniec wypżyczenia30 cze 2020        1208
-                                     620
                                     ... 
koniec wypżyczenia7 paź 2020            1
opłata za wypożyczenie:13 tys. €        1
opłata za wypożyczenie:577 tys. €       1
1,55 mln €                              1
koniec wypżyczenia5 cze 2019            1
Name: Operation, Length: 732, dtype: int64

Odejścia    9867
Nabytki     9616
Name: Bought/Sold, dtype: int64

200 tys. €     1307
300 tys. €     1176
400 tys. €     1051
250 tys. €     1038
100 tys. €      961
               ... 
1,75 mln €        2
75,00 mln €       2
1,25 mln €        2
80,00 mln €       1
825 tys. €        1
Name: PlayerMarketValue, Length: 124, dtype: int64

Anglia           2218
Włochy           1026
Turcja            932
Brazylia          848
Hiszpania         793
                 ... 
Wyspy Owcze         2
Mozambik            1
Bangladesz          1
Nigeria             1
Nowa Zelandia       1
Name: ClubAssignedLeagueNationality, Length: 94, dtype: int64

bez odstępnego                       5368
wypożyczenie                         4638
?                                    1758
koniec wypżyczenia30 cze 2020        1208
-                                     620
                                     ... 
koniec wypżyczenia7 paź 2020            1
opłata za wypożyczenie:13 tys. €        1
opłata za wypożyczenie:577 tys. €       1
1,55 mln €                              1
koniec wypżyczenia5 cze 2019            1
Name: TransferValue, Length: 732, dtype: int64

### Stworzenie dodatkowych kolumn określających ogólną pozycję zawodnika oraz przedział wiekowy

In [11]:
import random

def get_player_age_group(age):
    if 15 <= age < 16:
        return 'Junior młodszy'
    elif 16 <= age < 17:
        return 'Junior starszy'
    elif 17 <= age < 22:
        return 'Młody'
    elif 22 <= age < 30:
        return 'W sile wieku'
    elif 30 <= age < 37:
        return 'Doświadczony'
    elif 37 <= age < 42:
        return 'Zbliżający się do końca kariery'
    elif age >= 42:
        return 'Oldboy'
    else:
        return 'Inny'

transfer_df['PlayerGenericPosition'] = transfer_df['PlayerPosition'].map(lambda x: x.split(' ')[-1].capitalize())
transfer_df['PlayerAgeGroup'] = transfer_df['PlayerAge'].map(lambda x: get_player_age_group(random.randint(15,45) if x == '-' else int(x)))

display(transfer_df[['PlayerPosition', 'PlayerGenericPosition', 'PlayerAge', 'PlayerAgeGroup']])

,PlayerPosition,PlayerGenericPosition,PlayerAge,PlayerAgeGroup
0,Środkowy napastnik,Napastnik,19,Młody
1,Prawy obrońca,Obrońca,24,W sile wieku
2,Prawy napastnik,Napastnik,23,W sile wieku
3,Ofensywny pomocnik,Pomocnik,29,W sile wieku
4,Ofensywny pomocnik,Pomocnik,23,W sile wieku
...,...,...,...,...
27084,Defensywny pomocnik,Pomocnik,26,W sile wieku
27085,Lewy napastnik,Napastnik,27,W sile wieku
27107,Lewy napastnik,Napastnik,28,W sile wieku
27108,Defensywny pomocnik,Pomocnik,27,W sile wieku


### Wyczyszczenie danych z kolumny 'TransferDate'

In [12]:
def translate_months(month):
    if month == "sty": return "january"
    if month == "lut": return "february"
    if month == "mar": return "march"
    if month == "kwi": return "april"
    if month == "maj": return "may"
    if month == "cze": return "june"
    if month == "lip": return "july"
    if month == "sie": return "august"
    if month == "wrz": return "september"
    if month == "paź": return "october"
    if month == "lis": return "november"
    if month == "gru": return "december"


transfer_df = transfer_df[transfer_df['TransferDate'] != '0']

transfer_df['TransferDate'] = transfer_df['TransferDate'].astype("string")

transfer_df['TransferDate'] = transfer_df['TransferDate'].map(lambda x: datetime.strptime(x.split(" ")[0]+" "+translate_months(x.split(" ")[1])+" "+x.split(" ")[2], '%d %B %Y'))

transfer_df['TransferDate']= pd.to_datetime(transfer_df['TransferDate'])

transfer_df['TransferDate'].value_counts()

2021-07-01    2038
2020-07-01     702
2021-01-01     605
2021-08-31     550
2021-02-01     460
              ... 
2020-02-05       1
2017-07-11       1
2017-02-16       1
2017-11-16       1
2021-12-06       1
Name: TransferDate, Length: 809, dtype: int64

### Wyczyszczenie danych z kolumny 'PlayerMinutesPlayed'

In [13]:
transfer_df['PlayerMinutesPlayed'] = transfer_df['PlayerMinutesPlayed'].map(lambda x: x if len(x) == 1 else x[:-1])
transfer_df['PlayerMinutesPlayed'] = transfer_df['PlayerMinutesPlayed'].map(lambda x: x.replace(".", "").replace("-", ""))

transfer_df["PlayerMinutesPlayed"].value_counts()
transfer_df['PlayerMinutesPlayed'] = transfer_df['PlayerMinutesPlayed'].astype("int64")

### Wyczyszczenie danych z kolumny 'Operation'

In [17]:
transfer_df["Operation"].replace({"bez odstępnego": "Transfer", "-": "Transfer", "wypożyczenie": "Wypożyczenie", "draft": "Transfer"}, inplace=True)
transfer_df["Operation"].replace(["^koniec wypżyczenia[0-9]{1,2}[ ][a-ź]{3}[ ][0-9]{4}$"], ["Wypożyczenie"], regex=True, inplace=True)
transfer_df["Operation"].replace(["^opłata za wypożyczenie:.*$"], ["Wypożyczenie"], regex=True, inplace=True)
transfer_df["Operation"].replace(["^.*€$"], ["Transfer"], regex=True, inplace=True)

transfer_df = transfer_df[transfer_df["Operation"] != "?"]
transfer_df = transfer_df[transfer_df["Operation"] != '0']

transfer_df["Operation"].value_counts()

Wypożyczenie    9528
Transfer        8120
Name: Operation, dtype: int64

### Wyczyszczenie danych z kolumny 'Bought/Sold'

In [18]:
transfer_df["Bought/Sold"].replace({"Odejścia": "Sprzedany", "Nabytki": "Kupiony"}, inplace=True)
transfer_df['Bought/Sold'].value_counts()

Sprzedany    8968
Kupiony      8680
Name: Bought/Sold, dtype: int64

In [19]:
def human_readable_to_numeric(value):
    if value == 0:
        return 0
    elements = value.split(" ")
    base = float(elements[0].replace(',', '.'))
    if elements[1] == "mln":
        times = 1000000
    elif elements[1] == "tys.":
        times = 1000
    else:
        times = 1

    return base * times



transfer_df["PlayerMarketValue"].replace({"-": 0}, inplace=True)
transfer_df["PlayerMarketValue"] = transfer_df["PlayerMarketValue"].map(lambda x: human_readable_to_numeric(x))
transfer_df['PlayerMarketValue'] = transfer_df['PlayerMarketValue'].astype("int64")

display(transfer_df['PlayerMarketValue'].value_counts())

200000      1161
300000      1014
250000       924
400000       911
100000       871
            ... 
9500000        2
75000000       2
825000         1
80000000       1
1250000        1
Name: PlayerMarketValue, Length: 124, dtype: int64

### Wyczyszczenie danych z kolumny 'TransferValue'

In [22]:
def remove_text_about_loan_fee(value):
    if value == 0:
        return 0

    elements = value.split("opłata za wypożyczenie:")
    return elements[-1]


transfer_df["TransferValue"].replace({"bez odstępnego": 0, "-": 0, "wypożyczenie": 0, "draft": 0, "": 0}, inplace=True)
transfer_df["TransferValue"].replace(["^koniec wypżyczenia[0-9]{1,2}[ ][a-ź]{3}[ ][0-9]{4}$"], [0], regex=True, inplace=True)
transfer_df["TransferValue"] = transfer_df["TransferValue"].map(lambda x: remove_text_about_loan_fee(x))
transfer_df["TransferValue"] = transfer_df["TransferValue"].map(lambda x: human_readable_to_numeric(x))

transfer_df = transfer_df[transfer_df["TransferValue"] != "?"]
transfer_df = transfer_df[transfer_df['TransferValue'] != '0']

transfer_df['TransferValue'] = transfer_df['TransferValue'].astype("int64")

transfer_df["TransferValue"].value_counts()

0          15159
500000       123
1000000      118
2000000       99
1500000       82
           ...  
2900000        1
980000         1
112000         1
230000         1
479000         1
Name: TransferValue, Length: 409, dtype: int64

### Uzyskanie listy duplikatów oraz ich usunięcie

In [23]:
display(transfer_df[transfer_df.duplicated()])
transfer_df.drop_duplicates(keep='first', inplace=True)
display(transfer_df[transfer_df.duplicated()])

,Club,ClubLeagueLevel,ClubLeagueNationality,Player,PlayerAge,PlayerHeight,TransferDate,PlayerMatchesPlayed,PlayerGoals,PlayerAssists,PlayerOwnGoals,PlayerSubstitutionIn,PlayerSubstitutionOut,PlayerYellowCards,PlayerSecondYellowCards,PlayerRedCards,PlayerMinutesPlayed,PlayerPosition,PlayerPositionScut,PlayerNationality,Operation,Bought/Sold,PlayerMarketValue,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue,PlayerWithoutClub,PlayerGenericPosition,PlayerAgeGroup
1022,Manchester City,1,Anglia,Patrick Roberts,23,1.67,2021-08-31,181,26,38,0,84,59,6,0,0,9010,Prawy napastnik,PNP,Anglia,Wypożyczenie,Kupiony,1500000,FC Middlesbrough,2,Anglia,0,False,Napastnik,W sile wieku
5943,Fluminense Football Club,1,Brazylia,Reginaldo,28,1.93,2021-04-01,139,6,2,0,13,8,25,1,1,11319,Środkowy obrońca,ŚOB,Brazylia,Wypożyczenie,Sprzedany,350000,Clube de Regatas Brasil (AL),2,Brazylia,0,False,Obrońca,W sile wieku
9239,Atlanta United FC,1,Stany Zjednaczone,Rocco Ríos Novo,18,1.81,2019-01-07,22,0,0,0,1,0,1,0,0,1935,Bramkarz,BRK,Argentyna,Wypożyczenie,Kupiony,750000,Atlanta United 2,2,Stany Zjednaczone,0,False,Bramkarz,Młody
9266,Atlanta United FC,1,Stany Zjednaczone,Rocco Ríos Novo,18,1.81,2019-01-07,22,0,0,0,1,0,1,0,0,1935,Bramkarz,BRK,Argentyna,Wypożyczenie,Sprzedany,750000,Atlanta United 2,2,Stany Zjednaczone,0,False,Bramkarz,Młody
12539,FC Vaduz,1,Szwajcaria,Besart Bajrami,20,1.80,2022-01-01,36,1,0,0,13,11,3,0,0,2171,Ofensywny pomocnik,OPM,Serbia,Wypożyczenie,Sprzedany,0,FC Balzers,4,Szwajcaria,0,False,Pomocnik,Młody
13002,Gaziantep FK,1,Turcja,Mücahid Albayrak,29,1.80,2021-08-03,220,10,12,0,40,13,29,0,0,16863,Lewy obrońca,LOB,Turcja,Wypożyczenie,Kupiony,200000,Büyüksehir Belediye Erzurumspor,2,Turcja,0,False,Obrońca,W sile wieku
13785,Büyüksehir Belediye Erzurumspor,1,Turcja,Mücahid Albayrak,29,1.80,2021-08-03,220,10,12,0,40,13,29,0,0,16863,Lewy obrońca,LOB,Turcja,Wypożyczenie,Sprzedany,200000,Gaziantep FK,1,Turcja,0,False,Obrońca,W sile wieku
14271,Shanghai Shenhua,1,Chiny,Jianrong Zhu,29,1.85,2021-04-09,223,38,15,1,127,78,23,0,0,9693,Środkowy napastnik,ŚNP,Chiny,Wypożyczenie,Sprzedany,125000,Qingdao FC,1,Chiny,0,False,Napastnik,W sile wieku
14523,Qingdao FC,1,Chiny,Jianrong Zhu,29,1.85,2021-04-09,223,38,15,1,127,78,23,0,0,9693,Środkowy napastnik,ŚNP,Chiny,Wypożyczenie,Kupiony,125000,Shanghai Shenhua,1,Chiny,0,False,Napastnik,W sile wieku
16392,AC Horsens,1,Dania,Mikkel Qvist,27,2.03,2021-07-13,134,9,14,1,23,37,16,1,1,9593,Środkowy obrońca,ŚOB,Dania,Wypożyczenie,Kupiony,200000,KA Akureyri,1,Islandia,0,False,Obrońca,W sile wieku


,Club,ClubLeagueLevel,ClubLeagueNationality,Player,PlayerAge,PlayerHeight,TransferDate,PlayerMatchesPlayed,PlayerGoals,PlayerAssists,PlayerOwnGoals,PlayerSubstitutionIn,PlayerSubstitutionOut,PlayerYellowCards,PlayerSecondYellowCards,PlayerRedCards,PlayerMinutesPlayed,PlayerPosition,PlayerPositionScut,PlayerNationality,Operation,Bought/Sold,PlayerMarketValue,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue,PlayerWithoutClub,PlayerGenericPosition,PlayerAgeGroup


### Zapisanie wyczyszczonego zbioru danych do pliku csv

In [24]:
transfer_df.to_csv('csv/transfers-data-frame-cleared.csv', encoding='utf-8')